In [207]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import urllib.request
import urllib
import requests
import json

# url 리스트로 저장

In [279]:
headers = {'User-Agent': 'Mozilla/5.0'}

# 초기 세팅
url_list = []
wurl_list = []
i=1
previous_response = None

while True:
    # url 불러오기
    url = f'https://career.programmers.co.kr/api/job_positions?order=recent&page={i}&job_category_ids[]=1&job_category_ids[]=4&job_category_ids[]=25&job_category_ids[]=2&job_category_ids[]=3&job_category_ids[]=5&job_category_ids[]=11&job_category_ids[]=12&job_category_ids[]=92&job_category_ids[]=7&job_category_ids[]=16&job_category_ids[]=20&job_category_ids[]=9&job_category_ids[]=18&job_category_ids[]=59&job_category_ids[]=22&job_category_ids[]=26&job_category_ids[]=27&job_category_ids[]=10&job_category_ids[]=13&job_category_ids[]=17&job_category_ids[]=6&job_category_ids[]=125&job_category_ids[]=126&job_category_ids[]=127&job_category_ids[]=128&job_category_ids[]=129&job_category_ids[]=130&job_category_ids[]=158'

    res = requests.get(url,headers= headers)
    res_list = json.loads(res.text)

    # 이전 응답과 비교하여 정지 같으면 정지
    if res_list == previous_response:
        print("종료")
        break

    # 현재 응답을 이전 응답으로 저장
    previous_response = res_list

    for j in range(len(res_list['jobPositions'])):
        url_list.append('https://career.programmers.co.kr/'+res_list['jobPositions'][j]['url'])

    for w in range(len(res_list['wantedJobPositions'])):
        wurl_list.append(f"https://www.wanted.co.kr/wd/{res_list['wantedJobPositions'][w]['id']}?client_id=o673kBbqnYF6QVD4jKyTKhRP")

    i += 1

종료


In [280]:
len(url_list), len(wurl_list)

(736, 176)

# 채용 공고 데이터 프레임에 저장

In [ ]:
# 프로그래머스 공고 크롤링
job_data = []

for i in range(len(url_list)):
    id = re.findall(r'\d+',url_list[i])
    if len(id) > 1:
        print("에러", id)
        break

    url = f'https://career.programmers.co.kr/api/job_positions/{id[0]}'
    res = requests.get(url,headers= headers)
    job_json = json.loads(res.text)
    job_data.append(job_json)

# 데이터 프레임으로 변환
df = pd.json_normalize(job_data)


# df 저장
# df.to_csv('job_positions.csv', index=False, encoding = 'cp949')
df.to_csv('job_positions.csv', index=False, encoding='utf-8-sig')

df.info



In [ ]:
# 원티드 공고 크롤링
wjob_data = []

for i in range(len(wurl_list)):
    wid = re.findall(r'\d+',wurl_list[i])

    url = f'https://www.wanted.co.kr/api/chaos/jobs/v3/{wid[0]}/details?1736076088092='
    res = requests.get(url,headers= headers)
    wjob_json = json.loads(res.text)
    wjob_data.append(wjob_json)

# 데이터 프레임으로 변환
wdf = pd.json_normalize(wjob_data)


# wdf 저장
# wdf.to_csv('job_positions.csv', index=False, encoding = 'cp949')
wdf.to_csv('wjob_positions.csv', index=False, encoding='utf-8-sig')

wdf.info
